## Calcular o Tempo de Voto

Este notebook tem como objetivo calcular o tempo de voto de um eleitor. 
O primeiro passo é definir exatamente o que é um voto, dado que o log das urnas contém apenas uma sequência de eventos.

Na sequência, os votos são individualizados (um por linha) e o tempo de cada evento relevante é calculado.

## Importing libraries

In [28]:
import duckdb
import pandas as pd
import time

## Importing Data

In [29]:
cursor = duckdb.connect()

In [31]:
TABLE = "read_parquet('UFS_VOTE_EVENTS.parquet/*/*/*.parquet', hive_partitioning=True)"

In [32]:
source_data = f"""
    (
        SELECT
        *
        FROM {TABLE}
    ) AS source
"""

## Preparinga Data

### Criando um ID único para cada voto

Como heurística, vamos criar um id único para cada voto, que será determinado a partir de uma operação 'âncora'.

A operação servirá como marcação de que um voto foi iniciado e, todas as linhas entre uma operação âncora e a próxima, serão consideradas como um único voto.

In [33]:
ANCHOR_OPERATION = 'Aguardando digitação do título'
ZONE_GROUPS = [ (0, 100), (101, 200), (201, 300), (301, 400), (401, 500) ]

Após uma exploração dos LOGS, a operação escolhida foi 'AGUARDANDO DIGITAÇÃO DO TÍTULO', exatamente por ser o PRIMEIRO e OBRIAGTÓRIO passo para que um voto seja autorizado.

In [34]:
query_create_id = f"""
    (
        SELECT
            *,
            SUM(CASE WHEN event_description = '{ANCHOR_OPERATION}' THEN 1 ELSE 0 END) 
            OVER (PARTITION BY event_date, uf, filename ORDER BY event_timestamp) AS vote_id,
            
            CASE
                {
                    "".join(
                        [
                            f"WHEN zone_code::INT BETWEEN {min_zone} AND {max_zone} THEN '{min_zone}-{max_zone}' " 
                            for min_zone, max_zone in ZONE_GROUPS
                        ]
                    )
                }
            END AS zone_group

        FROM {source_data}
        WHERE 
        uf = '<uf>' 
        AND event_date = '<event_date>'
        AND zone_code::INT BETWEEN <zone_id_min> AND <zone_id_max>
    ) AS query_vote_id
"""

### Pivotando Timestamp dos eventos por id

Para calcular o tempo dos votos e dos eventos individuais que o compõem (biometria, voto) é necessário extrair o timestamp de cada evento.

In [35]:
timestamp_inicio_fim_voto = [
    f'''
        MAX(
            CASE WHEN event_description = 'Título digitado pelo mesário' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_titulo_digitado
    ''',
    f'''
        MAX(
            CASE WHEN event_description = 'O voto do eleitor foi computado' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_voto_computado
    '''
]

In [36]:
VOTE_EVENTS = [
    'Voto confirmado para [Conselheiro Distrital]',
    'Voto confirmado para [Deputado Distrital]',
    'Voto confirmado para [Deputado Estadual]',
    'Voto confirmado para [Deputado Federal]',
    'Voto confirmado para [Governador]',
    'Voto confirmado para [Prefeito]',
    'Voto confirmado para [Presidente]',
    'Voto confirmado para [Senador]',
]

timestamp_vote_events = [
    f'''
        MAX(
            CASE WHEN event_description = \'{event}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_voto_{event.replace("Voto confirmado para [", "").replace("]", "").lower().replace(' ', '_')}
    '''
    for event in VOTE_EVENTS
]

In [37]:
BIOMETRIA_TENTATIVAS = [
    'Solicita digital. Tentativa [1] de [4]',
    'Solicita digital. Tentativa [2] de [4]',
    'Solicita digital. Tentativa [3] de [4]',
    'Solicita digital. Tentativa [4] de [4]',
    'Solicitação de dado pessoal do eleitor para habilitação manual',
    'Eleitor foi habilitado'
]

timestamp_biometria_tentativas = [
    f'''
        MAX(
            CASE WHEN event_description = \'{event}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_biometria_{event.replace("Solicita digital. Tentativa [", "").replace("] de [4]", "").lower()}
    '''
    for event in BIOMETRIA_TENTATIVAS
    if event.startswith('Solicita digital')
] + [
    f'''
        MAX(
            CASE WHEN event_description = \'{BIOMETRIA_TENTATIVAS[-2]}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_biometria_manual
    '''
] + [
    f'''
        MAX(
            CASE WHEN event_description = \'{BIOMETRIA_TENTATIVAS[-1]}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_habilitacao_eleitor
    '''
]
    

In [38]:
query_pivot_timestamps = f"""(
    SELECT
        event_date, uf, filename, vote_id,
        
        MAX(city_code) AS city_code,
        MAX(zone_code) AS zone_code,
        MAX(zone_group) AS zone_group,
        MAX(section_code) AS section_code,

        SUM( (event_description='O voto do eleitor foi computado')::INT ) AS quantidade_votos_computados,
        SUM( (event_description ILIKE 'Solicita digital%')::INT ) AS quantidade_solicitacoes_biometria,
        SUM( (event_description ILIKE 'Voto confirmado para%')::INT ) AS quantidade_cargos_votados,
        MAX( (event_description='Solicitação de dado pessoal do eleitor para habilitação manual')::INT ) AS biometria_nao_funcionou,

        MIN( event_timestamp ) AS timestamp_primeiro_evento,

        {', '.join(timestamp_vote_events+timestamp_biometria_tentativas+timestamp_inicio_fim_voto)}
        
    FROM {query_create_id}
    GROUP BY event_date, uf, filename, vote_id
)
"""

### Construindo e Executando a query

Os arquivos parquet são particionados por DATA DO EVENTO, UF e GRUPO DE ZONA ELEITORAL por duas razões:

    - Facilitar a leitura dos dados posteriormente
    - Permitir a execução da query em partes, evitando a sobrecarga de memória ao processar todos os dados de uma vez

As ZONAS foram agrupadas em grupos de 100, esse número é empírico, pensado para abarcar a grande maioria das UFs em um único grupo, já que a grande maioria dos estados não pssui mais de 100 zonas eleitorais, e dividir as UFs mais populosas em grupos menores.

In [ ]:
ACCEPTED_DATES = [
    '2022-10-02', '2022-10-30', 
    '2022-10-03', '2022-10-31',
]
UFS = [
    'AC', 'AL', 'AM', 'AP', 
    'BA', 
    'CE', 'DF', 'ES', 'GO', 
    'MT', 'PA', 'PB', 'PE', 
    'MA',
    
    'MG', 'MS', 
    'PI', 'PR', 'RJ', 'RN', 
    'RO', 'RR', 'RS', 'SC', 
    'SE', 'SP', 'TO', 'ZZ'
]

PROCESSING_TIMES = []

for uf in UFS:
    for date in ACCEPTED_DATES:
        for zone_group in ZONE_GROUPS:

            
            query = F"""
                COPY 
                {
                    query_pivot_timestamps
                    .replace('<uf>', uf)
                    .replace('<event_date>', date)
                    .replace('<zone_id_min>', str(zone_group[0]))
                    .replace('<zone_id_max>', str(zone_group[1]))
                } 
                TO 'VOTES.parquet' 
                (FORMAT 'parquet', PARTITION_BY (event_date, uf, zone_group), OVERWRITE_OR_IGNORE 1);
            """
            
            print("Processing ", uf, date)
            tic = time.time()
            cursor.execute(query)
            toc = time.time()
            print(F"Time for {uf} {date} {zone_group}: {toc-tic}")

            PROCESSING_TIMES.append({
                'uf': uf,
                'date': date,
                'zone_group': zone_group,
                'time': toc-tic
            })

Salvando o resultado dos tempos de processamento.

In [42]:
PROCESSING_TIMES

# convert to pandas and save as csv
df_processing_times = pd.DataFrame(PROCESSING_TIMES)
df_processing_times.to_csv('processing_times.csv', index=False)